In [19]:
import pandas as pd
import numpy as np
import random
from flwr.simulation import run_simulation
from flwr.client import ClientApp
from flwr.server import ServerApp
from flwr.common import Context
import flwr as fl
from flwr.server import ServerAppComponents, ServerConfig

from sklearn.model_selection import train_test_split
from keras import layers, models, Input, regularizers, callbacks, optimizers

In [20]:
df1 = pd.DataFrame({
    "src_ip": [f"192.168.0.{i%3}" for i in range(20)],
    "dst_ip": [f"10.0.0.{i%2}" for i in range(20)],
    "timestamp": pd.date_range("2021-01-01", periods=20, freq="min").strftime("%d/%m/%Y %I:%M:%S %p"),
    "flow_id": list(range(20)),
    "f1": [random.randint(0, 100) for _ in range(20)],
    "f2": [random.randint(0, 100) for _ in range(20)],
    "f3": [random.randint(0, 100) for _ in range(20)],
    "f4": [random.randint(0, 100) for _ in range(20)],
    "f5": [random.randint(0, 100) for _ in range(20)],
    "f6": [random.randint(0, 100) for _ in range(20)],
    "class": ["Benign"] * 6 + ["bot"] * 8 + ["dos"] * 6,
    "label": [0] * 6 + [1] * 14
})
df2 = pd.DataFrame({
    "src_ip": [f"192.168.1.{i%4}" for i in range(20)],
    "dst_ip": [f"10.0.1.{i%3}" for i in range(20)],
    "timestamp": pd.date_range("2021-02-01", periods=20, freq="min").strftime("%d/%m/%Y %I:%M:%S %p"),
    "flow_id": list(range(20)),
    "f1": [random.randint(0, 100) for _ in range(20)],
    "f2": [random.randint(0, 100) for _ in range(20)],
    "f3": [random.randint(0, 100) for _ in range(20)],
    "f4": [random.randint(0, 100) for _ in range(20)],
    "f5": [random.randint(0, 100) for _ in range(20)],
    "f6": [random.randint(0, 100) for _ in range(20)],
    "class": ["Benign"] * 10 + ["bot"] * 10,
    "label": [0] * 10 + [1] * 10
})

df3 = pd.DataFrame({
    "src_ip": [f"192.168.1.{i%4}" for i in range(10)],
    "dst_ip": [f"10.0.1.{i%3}" for i in range(10)],
    "timestamp": pd.date_range("2021-02-01", periods=10, freq="min").strftime("%d/%m/%Y %I:%M:%S %p"),
    "flow_id": list(range(10)),
    "f1": [random.randint(0, 100) for _ in range(10)],
    "f2": [random.randint(0, 100) for _ in range(10)],
    "f3": [random.randint(0, 100) for _ in range(10)],
    "f4": [random.randint(0, 100) for _ in range(10)],
    "f5": [random.randint(0, 100) for _ in range(10)],
    "f6": [random.randint(0, 100) for _ in range(10)],
    "class": ["Benign"] * 5 + ["bot"] * 3 + ["dos"] * 2,
    "label": [0] * 5 + [1] * 5
})

In [21]:
def create_keras_model(input_shape, alpha = 0.001):
    LAMBD_2 = 0.001
    model = models.Sequential()
    
    model.add(layers.Conv1D(80, kernel_size=3,
                activation="relu", input_shape=(input_shape, 1), kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    # .L1L2(l1=LAMBD_1, l2=LAMBD_2)
    model.add(layers.Conv1D(80, 3, activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(80,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))

    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    return model

In [22]:
class FLClient(fl.client.NumPyClient):
    def __init__(self, context, X_train, X_val, y_train, y_val, model, num_local_epochs):

        self.context = context
        self.X_train = X_train
        self.X_val = X_val
        self.y_train = y_train
        self.y_val = y_val
        
        self.model = model
        self.num_local_epochs = num_local_epochs

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters):
        self.model.set_weights(parameters)
        
    def fit(self, parameters, config):

        lr=0.001
        self.model = create_keras_model(input_shape=self.input_dim, alpha=lr)
        self.set_parameters(parameters, config)

        history = self.model.fit(self.x_train, self.y_train,
                                epochs=config["local_epochs"],
                                batch_size=config["batch_size"],
                                validation_data=(self.x_val, self.y_val),  
                                verbose=0)

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.X_val, self.y_val, 2, verbose=0)
        return loss, len(self.X_val), {"accuracy": accuracy}

In [23]:
def generate_client_fn(data, labels):

    def client_fn(context: Context):

        client_id = int(context.node_config["partition-id"])
        print(f"==>> client_id: {client_id}")

        X_train, X_val, y_train, y_val = train_test_split(
            data[client_id], labels[client_id], test_size=0.1, random_state=1)

        print(f"==>> X_train: {X_train}")
        print(f"==>> X_val: {X_val}")
        
        
        model = create_keras_model(6)

        return FLClient(
            context, np.array(X_train), np.array(X_val), np.array(y_train), np.array(y_val), model, 1
        ).to_client()

    return client_fn


In [24]:
clients_data = [
    df1[["f1", "f2", "f3", "f4", "f5", "f6"]],
    df2[["f1", "f2", "f3", "f4", "f5", "f6"]]
]
# print(f"==>> clients_data: {clients_data}")
clients_labels = [
    df1["class"],
    df2["class"]
]
# print(f"==>> clients_labels: {clients_labels}")
client_app = ClientApp(client_fn=generate_client_fn(
    clients_data, clients_labels))

In [25]:
def get_evaluate_fn(x_test_server, y_test_server):
    def evaluate_fn(server_round: int, parameters, config):
        eval_model = create_keras_model(input_shape=6)
        eval_model.set_weights(parameters)

        test_loss, test_acc = eval_model.evaluate(x_test_server, y_test_server,
                                                  batch_size = 2)
        
        results_dict = {
            "test_loss": test_loss,
            "test_acc": test_acc,
            "round": server_round
        }

        return results_dict

    return evaluate_fn


In [26]:
def get_on_fit_config():

    def fit_config_fn(server_round: int):
        return {
            "lr": 0.001,
            "local_epochs": 1,
            "batch_size": 2,
        }

    return fit_config_fn

In [27]:
def generate_server_fn(data, labels):
    def server_fn(context: Context):
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
            min_fit_clients=len(clients_data),  # number of clients to sample for fit()
            fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
            min_evaluate_clients=0,  # number of clients to sample for evaluate()
            min_available_clients=len(clients_data),
            on_fit_config_fn=get_on_fit_config(),  # a function to execute to obtain the configuration to send to the clients during fit()
            evaluate_fn=get_evaluate_fn(data, labels),
        )
        
        return ServerAppComponents(
            strategy=strategy,
            config=ServerConfig(num_rounds=2)
        )

    return server_fn
server_app = ServerApp(server_fn=generate_server_fn(df3[["f1", "f2", "f3", "f4", "f5", "f6"]], df3["class"]))


In [ ]:
run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=2,
)
